In [124]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Fetched 337 kB in 1s (365 kB/s)
Reading package lists... Done


In [125]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [126]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
#looked at code from 22.2 act 3
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)
df.show(5)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [127]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')

In [128]:
#adding this cell to look at 22.2 activity 7 to converting date to a more workable date object
#before we work on next cell for average home price
spark.sql("""SELECT id,
   TO_DATE(date, 'MMMM dd, yyyy')
   AS date,
   date_built,
   price,
   bedrooms,
   bathrooms,
   sqft_living,
   sqft_lot,
   floors,
   waterfront,
   view
   FROM home_sales
   WHERE date IS NOT null""").show(10)

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [129]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
#got rounding help from https://stackoverflow.com/questions/53927574/how-to-round-decimal-in-scala-spark
spark.sql("SELECT year(date), round(AVG(price),2) FROM home_sales WHERE bedrooms = 4 GROUP BY year(date)").show()

+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           296363.88|
|      2019|            300263.7|
|      2020|           298353.78|
|      2021|           301819.44|
+----------+--------------------+



In [130]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("SELECT year(date), AVG(price) FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY year(date)").show()

+----------+------------------+
|year(date)|        avg(price)|
+----------+------------------+
|      2022| 292725.6931818182|
|      2019|287287.81696428574|
|      2020| 294204.1629104958|
|      2021| 294211.4595798854|
+----------+------------------+



In [131]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("SELECT year(date), round(AVG(price),2) FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living>=2000 GROUP BY year(date)").show()

+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           290242.99|
|      2019|           289859.14|
|      2020|           292289.09|
|      2021|           296330.96|
+----------+--------------------+



In [132]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
#the question in this cell doesnt make as much sense as the question in the module is more clear, which is as follows:
#What is the "view" rating for homes costing more than or equal to $350,000? Determine the run time for this query, and round off your answer to two decimal places.


start_time = time.time()

spark.sql("SELECT view, round(AVG(price),2) FROM home_sales WHERE price >=350000 GROUP BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  31|           399856.95|
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  34|           401419.75|
|  81|          1053472.79|
|  28|           402124.62|
|  76|          1058802.78|
|  26|           401506.97|
|  27|           399537.66|
|  44|           400598.05|
|  12|           401501.32|
|  91|          1137372.73|
|  22|           402022.68|
|  93|          1026006.06|
|  47|            398447.5|
|   1|           401044.25|
|  52|           733780.26|
|  13|           398917.98|
+----+--------------------+
only showing top 20 rows

--- 0.7571101188659668 seconds ---


In [133]:
# 7. Cache the the temporary table home_sales.
#module 22.3 act 6
spark.sql("cache table home_sales")

DataFrame[]

In [134]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [135]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT view, round(AVG(price),2) FROM home_sales WHERE price >=350000 GROUP BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  31|           399856.95|
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  34|           401419.75|
|  81|          1053472.79|
|  28|           402124.62|
|  76|          1058802.78|
|  26|           401506.97|
|  27|           399537.66|
|  44|           400598.05|
|  12|           401501.32|
|  91|          1137372.73|
|  22|           402022.68|
|  93|          1026006.06|
|  47|            398447.5|
|   1|           401044.25|
|  52|           733780.26|
|  13|           398917.98|
+----+--------------------+
only showing top 20 rows

--- 0.45096898078918457 seconds ---


In [136]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("homes_pq")

In [137]:
# 11. Read the parquet formatted data.
#module 22.3 act 3
#df.write.parquet('homes_parquet',mode='overwrite')
df=spark.read.parquet('homes_pq')

In [138]:
# 12. Create a temporary table for the parquet data.
df.createOrReplaceTempView('homes_pq_view')

In [139]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT view, round(AVG(price),2) FROM home_sales WHERE price >=350000 GROUP BY view").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  31|           399856.95|
|  85|          1056336.74|
|  65|           736679.93|
|  53|            755214.8|
|  78|          1080649.37|
|  34|           401419.75|
|  81|          1053472.79|
|  28|           402124.62|
|  76|          1058802.78|
|  26|           401506.97|
|  27|           399537.66|
|  44|           400598.05|
|  12|           401501.32|
|  91|          1137372.73|
|  22|           402022.68|
|  93|          1026006.06|
|  47|            398447.5|
|   1|           401044.25|
|  52|           733780.26|
|  13|           398917.98|
+----+--------------------+
only showing top 20 rows

--- 0.31664085388183594 seconds ---


In [140]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [141]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False